# 00_download_metadata.py

In [5]:
%load_ext autoreload

%autoreload 2
from isic_api import ISICApi
import os
import json
import csv
from tqdm.autonotebook import tqdm
with open('config.json') as json_file:
    data = json.load(json_file)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
api = ISICApi(username=None, password=None)
data_path = data["data_folder"]
# num_imgs = data["num_imgs"]
# for test I change it to 100, uncomment the above line to have 25000 image
num_imgs = 200
if not os.path.exists(data_path):
    os.makedirs(data_path)
imageList = []
count = 0
while len(imageList) < num_imgs:
    if count == 0:
        temp = api.getJson('images/?limit=' + str(num_imgs) +'&offset=0&sort=name')
        imageList = temp['results']
        next_page = temp['next'].split("images")[1]
    else:
        temp = api.getJson('images' + next_page)
        imageList = imageList + temp['results']
        next_page = temp['next'].split("images")[1]
    count+=1

In [46]:
# %%            
print('Fetching metadata for %s images' % len(imageList))
imageDetails = []
for image in tqdm(imageList):
    # Fetch the full image details
    imageDetail = api.getJson('images/%s' % image['isic_id'])
    imageDetails.append(imageDetail)

Fetching metadata for 200 images


  0%|          | 0/200 [00:00<?, ?it/s]

In [47]:
# Determine the union of all image metadata fields
metadataFields = set(
        field
        for imageDetail in imageDetails
        for field in imageDetail['metadata']['clinical'].keys()
    )


metadataFields = ['isic_id'] + sorted(metadataFields)
outputFileName = "meta"
#%%
outputFilePath = os.path.join(data_path, outputFileName)
# Write the metadata to a CSV
print('Writing metadata to CSV: %s' % outputFileName+'.csv')
with open(outputFilePath+'.csv', 'w') as outputStream:
    csvWriter = csv.DictWriter(outputStream, metadataFields)
    csvWriter.writeheader()
    for imageDetail in imageDetails:
        rowDict = imageDetail['metadata']['clinical'].copy()
        rowDict['isic_id'] = imageDetail['isic_id']
        csvWriter.writerow(rowDict)

Writing metadata to CSV: meta.csv


# 01_download_imgs.py

In [57]:
from isic_api import ISICApi
import os
import json
import requests
with open('config.json') as json_file:
    data = json.load(json_file)

In [65]:
api = ISICApi(username=None, password=None)
data_path = data["data_folder"]
# num_imgs = data["num_imgs"]
num_imgs = 200
#%%
savePath = os.path.join(data_path, 'raw')

if not os.path.exists(savePath):
    os.makedirs(savePath)
start_offset = 0
#%%

for i in range(int(num_imgs/50)):
    if i == 0:
        temp = api.getJson('images/?limit=50')
        imageList = temp['results']
        next_page = temp['next'].split("images")[1]
    else:
        temp = api.getJson('images' + next_page)
        imageList = temp['results']
        next_page = temp['next'].split("images")[1]
    print('Downloading %s images' % len(imageList))
    
    for image in imageList:
        url = image['files']['full']['url']
        img_data = requests.get(url).content
        imageFileOutputPath = os.path.join(savePath, '%s.jpg' % image['isic_id'])
        with open(imageFileOutputPath, 'wb') as imageFileOutputStream:
            imageFileOutputStream.write(img_data)

# 02_sort_imgs.py

In [69]:
import cv2
import csv
import numpy as np
from tqdm import tqdm
from shutil import copyfile
from os.path import join as oj
from isic_api import ISICApi
import os
from os.path import join as oj
import json
import csv
with open('config.json') as json_file:
    data = json.load(json_file)

In [70]:
data_path = data["data_folder"]
img_path = os.path.join(data_path, "raw")
processed_path = os.path.join(data_path, "processed")
segmentation_path = os.path.join(data_path, "segmentation")
benign_path = os.path.join(processed_path, "no_cancer")
malignant_path = os.path.join(processed_path, "cancer")
os.makedirs(processed_path,exist_ok = True)
os.makedirs(benign_path,exist_ok = True)
os.makedirs(segmentation_path,exist_ok = True)
os.makedirs(malignant_path,exist_ok = True)
#%%

In [71]:
list_of_meta = []
from PIL import Image
with open(oj(data_path, "meta.csv"), newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    next(spamreader)
    for row in spamreader:
        list_of_meta.append(row)    

In [75]:
#%%
list_benign_files = []
for line in list_of_meta:
    if len(line) > 0 and line[4] == 'benign':
        list_benign_files.append(line[0] + ".jpg")
list_mal_files = []
for line in list_of_meta:
    if len(line) > 0 and line[4] == 'malignant':
        list_mal_files.append(line[0] + ".jpg")
#%%

In [79]:
def resize_and_save(my_list, my_folder):
    for i,file_name in tqdm(enumerate(my_list)):
        try:
            img = cv2.imread(oj(img_path, file_name))
            resized_img = cv2.resize(img, (299, 299))  # Resize the image to (299, 299)
            cv2.imwrite(oj(my_folder, file_name), resized_img)
        except:
            print(file_name)
resize_and_save(list_mal_files, malignant_path)
resize_and_save(list_benign_files, benign_path)

12it [00:00, 26.05it/s]
172it [00:09, 18.38it/s] 


# 03_calculate_pretrained.py

In [82]:
import torch
import torchvision
import torchvision.datasets as datasets
import sys
import numpy as np
import torch.utils.data as utils
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pickle as pkl
from os.path import join as oj
import os
from torch.utils.data import Subset
import csv
import numpy as np
sys.path.append('../../src/')
from skimage.morphology import dilation
import cd
from shutil import copyfile
from os.path import join as oj
from PIL import Image
from tqdm import tqdm
from skimage.color import rgb2gray
import torchvision.models as models
from torch import nn    
from torch.nn import AdaptiveAvgPool2d
import json
with open('config.json') as json_file:
    data = json.load(json_file)

In [89]:
data_path = data["data_folder"]
processed_path = os.path.join(data_path, "processed")
benign_path = os.path.join(processed_path, "no_cancer")
malignant_path = os.path.join(processed_path, "cancer")
feature_path = os.path.join(data_path, "calculated_features")
segmentation_path = os.path.join(data_path, "segmentation")
os.makedirs(feature_path,exist_ok = True)
#%%
# used for converting to the range VGG16 is used to
mean = np.asarray([0.485, 0.456, 0.406])
std = np.asarray([0.229, 0.224, 0.225])

device = torch.device("cuda")
#%%


list_of_img_names = os.listdir(benign_path)


model = models.vgg16(pretrained=True).to(device).eval()

img_features = np.empty((len(list_of_img_names), 25088))
cd_features = -np.ones((len(list_of_img_names), 2, 25088)) # rel, irrel
avg_layer = torch.nn.AdaptiveAvgPool2d((7,7))
from skimage.morphology import square
my_square = square(20)
with torch.no_grad():
    for i in tqdm(range(len(list_of_img_names))):
        if not(list_of_img_names[i].startswith('.')):
            img = Image.open(oj(benign_path, list_of_img_names[i]))
            img_torch = torch.from_numpy(((np.asarray(img)/255.0 -mean)/std).swapaxes(0,2).swapaxes(1,2))[None,:].cuda().float()
            img.close()
            img_features[i] = avg_layer(model.features(img_torch)).view(-1).cpu().numpy()
            if os.path.isfile(oj(segmentation_path, list_of_img_names[i])):
                seg = Image.open(oj(segmentation_path, list_of_img_names[i]))
                blob =  dilation((np.asarray(seg)[:,:, 0] > 100).astype(np.uint8),my_square).astype(np.float32)
                
                rel, irrel =cd.cd_vgg_features(blob, img_torch, model)
                cd_features[i, 0] = rel[0].cpu().numpy()
                cd_features[i, 1] = irrel[0].cpu().numpy()



with open(oj(feature_path, "not_cancer.npy"), 'wb') as f:
    np.save(f, img_features)
with open(oj(feature_path, "not_cancer_cd.npy"), 'wb') as f:
    np.save(f, cd_features)
 


list_of_img_names = os.listdir(malignant_path)
img_features = np.empty((len(list_of_img_names), 25088))
with torch.no_grad():
    for i in tqdm(range(len(list_of_img_names))):
        if not(list_of_img_names[i].startswith('.')):
            img = Image.open(oj(malignant_path, list_of_img_names[i]))
            img_torch = torch.from_numpy(((np.asarray(img)/255.0 -mean)/std).swapaxes(0,2).swapaxes(1,2))[None,:].cuda().float()
            img.close()
            img_features[i] = avg_layer(model.features(img_torch)).view(-1).cpu().numpy()
with open(oj(feature_path, "cancer.npy"), 'wb') as f:
    np.save(f, img_features)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 187.19it/s]


# train_saliency.py

In [93]:
import torch
import sys
import numpy as np
import pickle as pkl
from os.path import join as oj

import torch.optim as optim
import os
from torch.utils.data import TensorDataset, ConcatDataset
import argparse
from PIL import Image
from tqdm import tqdm
from torch import nn
from numpy.random import randint
import torchvision.models as models
import time
import copy
sys.path.append('../../src/')
import gc
from score_funcs import  gradient_sum
import json
with open('config.json') as json_file:
    data = json.load(json_file)

In [122]:
model_path = os.path.join(data["model_folder"], "ISIC_saliency_new")
if not os.path.exists(model_path):
    os.makedirs(model_path)
print(model_path)
data_path =data["data_folder"]

seg_path  = oj(data_path, "segmentation")
not_cancer_path = oj(data_path, "processed/no_cancer")
cancer_path = oj(data_path, "processed/cancer")
 
 
mean = np.asarray([0.485, 0.456, 0.406])
std = np.asarray([0.229, 0.224, 0.225])


description='PyTorch MNIST Example'
batch_size = 16
epochs = 5
momentum = 0.9
seed = 42
lr = 1e-5
regularizer_rate = 0.0
device = torch.device(0)
torch.manual_seed(seed);
model = models.vgg16(pretrained=True)

model.classifier[-1] = nn.Linear(4096, 2)
model = model.to(device)
params_to_update = model.classifier.parameters()

../../models/ISIC_saliency_new


/home/ida01/mirzaei/anaconda3/envs/CDEP/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ida01/mirzaei/anaconda3/envs/CDEP/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [123]:
def load_folder(path):
    list_files= os.listdir(path)
    num_files = len(list_files)
    imgs_np = np.empty((num_files,  299, 299,3))
    for i in tqdm(range(num_files)):
        try:
            img = Image.open(oj(path, list_files[i]))
            imgs_np[i] = np.asarray(img)/255.0
            
            img.close()
        except:
            print(i)
    return imgs_np

def load_seg(path, orig_path):
    list_files= os.listdir(orig_path)
    num_files = len(list_files)
    imgs_np = np.zeros((num_files,  299, 299), dtype = bool)
    for i in tqdm(range(num_files)):
        if os.path.isfile(oj(path,  list_files[i])):
            img = Image.open(oj(path, list_files[i]))
            imgs_np[i] = np.asarray(img)[:,:,0] > 100
            img.close()
    return imgs_np

In [124]:
cancer_set = load_folder(cancer_path)
cancer_set -= mean[None, None, :]
cancer_set /= std[None, None, :]
not_cancer_set = load_folder(not_cancer_path)
not_cancer_set -= mean[None, None, :]
not_cancer_set /= std[None, None, :]
seg_set = load_seg(seg_path, not_cancer_path)

cancer_targets = np.ones((cancer_set.shape[0])).astype(np.int64)
not_cancer_targets = np.zeros((not_cancer_set.shape[0])).astype(np.int64)
not_cancer_dataset = TensorDataset(torch.from_numpy(not_cancer_set.swapaxes(1,3).swapaxes(2,3)).float(), torch.from_numpy(not_cancer_targets),torch.from_numpy(seg_set))
del not_cancer_set
del seg_set

cancer_dataset = TensorDataset(torch.from_numpy(cancer_set.swapaxes(1,3).swapaxes(2,2)).float(), torch.from_numpy(cancer_targets),torch.from_numpy(np.zeros((len(cancer_set), 299, 299), dtype = bool)))
del cancer_set

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 613.98it/s]


1


 32%|███████████████████████████████████████████████████████▋                                                                                                                       | 55/173 [00:00<00:00, 548.76it/s]

1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 173/173 [00:00<00:00, 178810.89it/s]


In [125]:
gc.collect()
complete_dataset = ConcatDataset((not_cancer_dataset, cancer_dataset ))
num_total = len(complete_dataset)
num_train = int(0.8 * num_total)
num_val = int(0.1 * num_total)
num_test = num_total - num_train - num_val
torch.manual_seed(seed);
train_dataset, test_dataset, val_dataset= torch.utils.data.random_split(complete_dataset, [num_train, num_test, num_val])
datasets = {'train' : train_dataset, 'test':test_dataset, 'val': val_dataset}
dataset_sizes = {'train' : len(train_dataset), 'test':len(test_dataset), 'val': len(val_dataset)}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test','val']}



       

cancer_ratio =len(cancer_dataset)/len(complete_dataset)


not_cancer_ratio = 1- cancer_ratio
cancer_weight = 1/cancer_ratio
not_cancer_weight = 1/ not_cancer_ratio
weights = np.asarray([not_cancer_weight, cancer_weight])
weights /= weights.sum()
weights = torch.tensor(weights).to(device)

criterion = nn.CrossEntropyLoss(weight = weights.double().float())


optimizer_ft = optim.SGD(params_to_update, lr=lr, momentum=momentum)

In [126]:
def train_model(model,dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    val_acc_history = []
    val_loss_history = []
    train_loss_history = []
    
    train_acc_history = []
    train_cd_history= []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10.0
    patience = 3
    cur_patience = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                optimizer.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_loss_cd = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, (inputs, labels, seg) in tqdm(enumerate(dataloaders[phase])):
    
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                seg = seg.to(device)
                

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # need to do calc beforehand because we do need the gradients
                    if phase == 'train' and regularizer_rate !=0:
                        inputs.requires_grad = True
                        add_loss = gradient_sum(inputs, labels, seg, model, criterion)  
                        if add_loss!=0:
                            (regularizer_rate*add_loss).backward()
                            optimizer.step()
                        #print(torch.cuda.memory_allocated()/(np.power(10,9)))
                        optimizer.zero_grad()   
                        running_loss_cd +=add_loss.item() * inputs.size(0)
     
                        #inputs.require_grad = False
                   
                    
                         
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        (loss).backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_cd_loss = running_loss_cd / dataset_sizes[phase]
       
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
  

            print('{} Loss: {:.4f} Acc: {:.4f} CD Loss : {:.4f}'.format(
                phase, epoch_loss, epoch_acc, epoch_cd_loss))

            # deep copy the model
            if phase == 'val':
                val_acc_history.append(epoch_acc.item())
                val_loss_history.append(epoch_loss)
            if phase == 'train':
                train_loss_history.append(epoch_loss)
                train_cd_history.append(epoch_cd_loss)
                train_acc_history.append(epoch_acc.item())
                

                
            if phase == 'val':
                if epoch_loss < best_loss:
            
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    cur_patience = 0
                else:
                    cur_patience+=1
        if cur_patience >= patience:
            break
             

 

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights

  
    hist_dict = {}
    hist_dict['val_acc_history'] = val_acc_history
    hist_dict['val_loss_history'] = val_loss_history
    
    hist_dict['train_acc_history'] = train_acc_history

    hist_dict['train_loss_history'] = val_loss_history
    hist_dict['train_cd_history'] = train_cd_history
    model.load_state_dict(best_model_wts)
    return model,hist_dict 
    

In [128]:
model, hist_dict = train_model(model, dataloaders, criterion, optimizer_ft, num_epochs=epochs)
pid = ''.join(["%s" % randint(0, 9) for num in range(0, 20)])
torch.save(model.classifier.state_dict(),oj(model_path, pid + ".pt"))

hist_dict['pid'] = pid
hist_dict['regublarizer_rate'] = regularizer_rate
hist_dict['seed'] = seed
hist_dict['batch_size'] = batch_size
hist_dict['learning_rate'] = lr
hist_dict['momentum'] = momentum
pkl.dump(hist_dict, open(os.path.join(model_path , pid +  '.pkl'), 'wb'))

Epoch 0/4
----------


10it [00:01,  9.99it/s]

train Loss: 0.6560 Acc: 0.7838 CD Loss : 0.0000



2it [00:00, 15.04it/s]

val Loss: 0.6661 Acc: 0.9444 CD Loss : 0.0000
Epoch 1/4
----------


...

val Loss: 0.6492 Acc: 0.9444 CD Loss : 0.0000
Epoch 4/4
----------



10it [00:01,  9.82it/s]

train Loss: 0.5592 Acc: 0.8784 CD Loss : 0.0000



2it [00:00, 15.26it/s]


val Loss: 0.6388 Acc: 0.9444 CD Loss : 0.0000
Training complete in 0m 7s
Best val loss: 0.638806


# train.py

In [130]:
import torch
from torch import nn
import torch.optim as optim
import torchvision.models as models
import sys
import pickle as pkl
import os
import argparse
from numpy.random import randint
import time
import copy
from tqdm import tqdm
sys.path.append('../../src')
import utils
import cd
import json
torch.backends.cudnn.deterministic = True #this makes results reproducible. 
with open('config.json') as json_file:
    data = json.load(json_file)

In [139]:
model_path = os.path.join(data["model_folder"], "ISIC_new")
if not os.path.exists(model_path):
    os.makedirs(model_path)
dataset_path =os.path.join(data["data_folder"],"calculated_features")

 
# Training settings
description='ISIC Skin cancer for CDEP'
batch_size = 32
epochs = 100
num_epochs = epochs
lr = 1e-2
momentum = 0.9
seed = 42
regularizer_rate = 0.0
device = torch.device(0)

torch.manual_seed(seed);
model = models.vgg16(pretrained=True)
model.classifier[-1] = nn.Linear(4096, 2)
model = model.classifier.to(device)
datasets, weights = utils.load_precalculated_dataset(dataset_path)
if regularizer_rate ==-1: # -1 means that we train only on data with no patches
    datasets['train'] = datasets['train_no_patches']

dataset_sizes= {x:len(datasets[x]) for x in datasets.keys()}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4)
              for x in datasets.keys()}





weights = torch.tensor(weights).to(device)

params_to_update = model.parameters()

criterion = nn.CrossEntropyLoss(weight = weights.double().float())


optimizer_ft = optim.SGD(params_to_update, lr=lr, momentum=momentum)

/home/ida01/mirzaei/anaconda3/envs/CDEP/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ida01/mirzaei/anaconda3/envs/CDEP/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [140]:
def train_model(model,dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    val_acc_history = []
    val_loss_history = []
    train_loss_history = []
    
    train_acc_history = []
    train_cd_history= []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1000.0
    patience = 3
    cur_patience = 0


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                optimizer.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_loss_cd = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, (inputs, labels, cd_features) in tqdm(enumerate(dataloaders[phase])):
    
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                cd_features = cd_features.to(device)
                

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                    
                        add_loss = torch.zeros(1,).cuda()
                        if regularizer_rate > 0:
                        
                            mask  = (cd_features[:, 0,0] != -1).bool()
                            if mask.any():
                                rel, irrel = cd.cd_vgg_classifier(cd_features[:,0], cd_features[:,1], inputs, model)
                   
                                cur_cd_loss = torch.nn.functional.softmax(torch.stack((rel[:,0].masked_select(mask),irrel[:,0].masked_select(mask)), dim =1), dim = 1)[:,0].mean() 
                                cur_cd_loss +=torch.nn.functional.softmax(torch.stack((rel[:,1].masked_select(mask),irrel[:,1].masked_select(mask)), dim =1), dim = 1)[:,0].mean() 
                                add_loss = cur_cd_loss/2

                        (loss+regularizer_rate*add_loss).backward()
                        # print how much memory is used
                        #print(torch.cuda.memory_allocated()/(np.power(10,9)))
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_loss_cd +=add_loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_cd_loss = running_loss_cd / dataset_sizes[phase]
       
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
  
            
            print('{} Loss: {:.4f} Acc: {:.4f} CD Loss : {:.4f}'.format(
                phase, epoch_loss, epoch_acc, epoch_cd_loss))

            # deep copy the model
            if phase == 'val':
                val_acc_history.append(epoch_acc.item())
                val_loss_history.append(epoch_loss)
            if phase == 'train':
                train_loss_history.append(epoch_loss)
                train_cd_history.append(epoch_cd_loss)
                train_acc_history.append(epoch_acc.item())
                
            if phase == 'val':
                if epoch_loss < best_loss:
            
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    cur_patience = 0
                else:
                    cur_patience+=1
        if cur_patience >= patience:
            break
             
                

 

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))



  
    hist_dict = {}
    hist_dict['val_acc_history'] = val_acc_history
    hist_dict['val_loss_history'] = val_loss_history
    hist_dict['train_acc_history'] = train_acc_history
    hist_dict['train_loss_history'] = val_loss_history
    hist_dict['train_cd_history'] = train_cd_history
    model.load_state_dict(best_model_wts)
    return model,hist_dict 

In [141]:
model, hist_dict = train_model(model, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

hist_dict['AUC (no patches)'],hist_dict['F1 score (no patches)'] =utils.get_auc_f1(model, datasets['test_no_patches'])

hist_dict['AUC (patches)'],hist_dict['F1 score (patches)'] =utils.get_auc_f1(model, datasets['test'])



pid = ''.join(["%s" % randint(0, 9) for num in range(0, 20)])
#torch.save(model.state_dict(),oj(model_path, pid + ".pt"))

hist_dict['pid'] = pid
hist_dict['regularizer_rate'] = regularizer_rate
hist_dict['seed'] = seed
hist_dict['batch_size'] = batch_size
hist_dict['learning_rate'] = lr
hist_dict['momentum'] = momentum
pkl.dump(hist_dict, open(os.path.join(model_path , pid +  '.pkl'), 'wb'))

Epoch 0/99
----------


5it [00:00, 28.94it/s]

train Loss: nan Acc: 0.6216 CD Loss : 0.0000



1it [00:00, 11.31it/s]

val Loss: nan Acc: 0.9444 CD Loss : 0.0000
Epoch 1/99
----------


...

val Loss: nan Acc: 0.9444 CD Loss : 0.0000
Epoch 2/99
----------



5it [00:00, 29.75it/s]

train Loss: nan Acc: 0.9324 CD Loss : 0.0000



1it [00:00, 12.77it/s]

val Loss: nan Acc: 0.9444 CD Loss : 0.0000
Training complete in 0m 1s
Best val loss: 1000.000000


# 04_train_all.py

you cant run this in notebook.
first you should change train_saliency.py files base in what I write in this notebook and then run that from terminal